Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [3]:
from random import choices
from dataclasses import dataclass
from copy import copy
from random import random, choice, randint, sample
import numpy
from tqdm import tqdm

import lab9_lib

ModuleNotFoundError: No module named 'lab9_lib'

In [ ]:
INSTANCES = (1,2,5,10)
LOCI_GENOMES = 1000

NUM_PROB = 20
PROB_REP = 10

OFFSPRING_SIZE = 100
POPULATION_SIZE = 500
MAX_ITERATIONS = 1000
TOURNAMENT_SIZE = 20

@dataclass
class Individual:
    genotype: list[bool]
    fitness: float

In [ ]:

def randMutation(individual: Individual, n = 1) -> Individual:
    offspring = copy(individual)
    positions = sample(range(0, LOCI_GENOMES), n)
    for pos in positions:
        offspring.genotype[pos] = not offspring.genotype[pos]
        offspring.fitness = None
    return offspring

def oneCutXover(individual1: Individual, individual2: Individual, cut_point) -> Individual:
    assert len(individual1.genotype) == len(individual2.genotype)
    assert cut_point < len(individual1.genotype)
    offspring = Individual(genotype = individual1.genotype[:cut_point] + individual2.genotype[cut_point:], fitness = None)
    assert len(offspring.genotype) == LOCI_GENOMES
    return offspring

def oneRandCutXover(individual1: Individual, individual2: Individual) -> Individual:
    cut_point = randint(0, LOCI_GENOMES-1)
    return oneCutXover(individual1, individual2, cut_point)

def andXover(individual1: Individual, individual2: Individual) -> Individual:
    offspring = Individual(genotype = [x and y for x,y in zip(individual1.genotype, individual2.genotype)], fitness = None)
    assert len(offspring.genotype) == LOCI_GENOMES
    return offspring

def orXover(individual1: Individual, individual2: Individual) -> Individual:
    offspring = Individual(genotype = [x or y for x,y in zip(individual1.genotype, individual2.genotype)], fitness = None)
    assert len(offspring.genotype) == LOCI_GENOMES
    return offspring

def xorXover(individual1: Individual, individual2: Individual) -> Individual:
    offspring = Individual(genotype = [x ^ y for x,y in zip(individual1.genotype, individual2.genotype)], fitness = None)
    assert len(offspring.genotype) == LOCI_GENOMES
    return offspring

def select_parent(population):
    pool = [choice(population) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

In [ ]:
def genPopulation(fitness, sorted = False):
    population = []
    for _ in range(POPULATION_SIZE):
        ind = Individual(
            genotype=[choice((False, True)) for _ in range(LOCI_GENOMES)],
            fitness=None
        )
        
        ind.fitness = fitness(ind.genotype)
        population.append(ind)
        
        if sorted:
            population.sort(key=lambda i: i.fitness, reverse= True)
    return population
    

In [ ]:
for instance in INSTANCES:
    probabilities = numpy.linspace(0, 1, NUM_PROB)
    prob_Rep = {f'{key:.2%}': 0 for key in probabilities}
    for _ in range(PROB_REP):
        min_calls = -1
        best_prob = ""
        for probability in probabilities:
            fitness = lab9_lib.make_problem(instance)
            population = genPopulation(fitness)
            solution = False
            for generation in range(MAX_ITERATIONS):
                offspring = list()
                for counter in range(OFFSPRING_SIZE):
                    if random() < probability:
                        p = select_parent(population)
                        o = randMutation(p)
                    else:
                        p1 = select_parent(population)
                        p2 = select_parent(population)
                        #o = oneRandCutXover(p1, p2)
                        o = orXover(p1, p2)
                    o.fitness = fitness(o.genotype)
                    offspring.append(o)
                    if o.fitness >= 1.0:
                        solution = True
                        break
                population.extend(offspring)
                population.sort(key=lambda i: i.fitness, reverse=True)
                population = population[:POPULATION_SIZE]
                if solution:
                    #print(f'Prob: {probability}, calls: {fitness.calls}')
                    if min_calls < 0:
                        min_calls = fitness.calls
                        best_prob = f'{probability:.2%}'
                    elif fitness.calls < min_calls:
                        min_calls = fitness.calls
                        best_prob = f'{probability:.2%}'
                    break
            if min_calls < 0:
                        min_calls = fitness.calls
                        best_prob = f'{probability:.2%}'
        prob_Rep[best_prob] = prob_Rep[best_prob] + 1
    
    print(f'instance: {instance}')
    print(prob_Rep)
